In [24]:
import pandas as pd
import numpy as np
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt 
import os
#import more_itertools as mit

In [25]:

FilePathList = "/Users/timmo/Documents/RapfLab/code/blanktest" #sets filepath for initial file/mass list. May not be necessary to keep this way
                                                            #but sets path for list 
FileType="csv" #sets filetype
MassNumbersList=[] #creates an empty list which will be filled by the numbers in the mass list file
os. chdir(FilePathList) #sets working directory to FilePathList
CurrFile = pd.read_csv("IncompCombinedmasslist.csv") #reads the file masslist.csv from the filepath --> since we only want to read one file was easier to just specify file
#print(CurrFile)
MassNumberNames=[]
#######################################


for i in (range(len(CurrFile))):
    MassNumbersList.append(CurrFile['mz_list'][i]) #appends value number i from masslist.csv in the colum mz_list
    MassNumberNames.append(CurrFile['Species'][i])
print(len(MassNumberNames))
print(len(MassNumbersList))

31
31


In [26]:
FilePathList = "/Users/timmo/Documents/RapfLab/code/blanktest/Processed"
FileType="csv" #sets filetype
os. chdir(FilePathList)
Blanks=[]
CurrFile = pd.read_csv("stdev_blank.csv")
for i in (range(len(CurrFile))):
    Blanks.append(CurrFile['standard deviation'][i])

print(Blanks)

[19.209065067698177, 5690.004185725135, 16.605473612619626, 7.893490833541339, 32.44048706088627, 84.84940365387688, 228.18626315399004, 12.211391031353983, 263592.9262316618, 0.7410382684473654, 0.6352892883891966, 2533.711277811056, 2533.711277811056, 1965.9401311667648, 922.0004925243582, 161.34009216734628, 477.7634631228077, 12.234142871423336, 2644.5399861517385, 15.225910554882653, 5.878238287603637, 2496.3233052519627, 296.3537047911188, 18102.85033870692, 43061.816144475626, 7.714204092211952, 1526.9312828778422, 1356.6377609997623, 14437.733445379934, 35540.05492042522, 902.8607228872158]


In [27]:

FilePath = "/Users/timmo/Documents/RapfLab/code/blanktest/files" #Second file path for where the actual files are located
os. chdir(FilePath) #updates file path to the folder with actual files for processing
AllFiles=pd.DataFrame()
FileType="csv" #what filetype are the files. Will probably be csv
files = sorted(glob.glob("*"+FileType)) #gets all files of chosen filetype
num_files = len(files) #how many files are you reading in
OutFolder = "Processed" #folder where processed files will be saved
outname = "" #part of file name. Will be used later
##############################
print(files)
ErrorRange = 0.00005 #error range. Will be +- to get range and see if values fit in range. 25 ppm error range     
mzlistRep=[]



for entry in range(num_files): #goes through all files
    name = files[entry] 
    CurrFile = pd.read_csv(files[entry],skiprows=1) #reads files
    FullSplit = name.split(".") #gets all parts of the file name
    NameList=(str(FullSplit[0]))
    mzlistRep=[]
    ProcessedX=[] #values for X(Thompsons) will be saved here
    ProcessedY=[] #values for Y(Counts) will be saved here
    PPM=[] #PPM values will be saved here
    names=[]
    BlankRep=[]
    Detected=[]
    #gets the third part of the name (0,1,[2]). Adds it to the name list
    for mass in range(len(MassNumbersList)): #goes through each mast in the MassesList
        MassHigh=((MassNumbersList[mass])+(MassNumbersList[mass]*ErrorRange)) #upper range of Mass
        MassLow=((MassNumbersList[mass])-(MassNumbersList[mass]*ErrorRange)) #lower range of Mass
        fixed=CurrFile.drop(columns="#Point") #drops point column
        fit=fixed["X(Thompsons)"].between(MassLow, MassHigh, inclusive = True) #checks if value for X(Thompson) fits between mass low and mass high
        ####Part two of function
        for i in range(len(fit)): #checks if values in fit are true or false
             if fit[i]== True: #if True then appends values
                    ProcessedX.append(fixed["X(Thompsons)"][i])
                    ProcessedY.append(fixed["Y(Counts)"][i])
                    PPM.append(((abs((fixed["X(Thompsons)"][i])-MassNumbersList[mass]))/MassNumbersList[mass])*1000000)
                    mzlistRep.append(MassNumbersList[mass])
                    names.append(MassNumberNames[mass])
                    BlankRep.append(Blanks[mass])
                    if fixed["Y(Counts)"][i].between(0, 3*Blanks[mass]):
                        Detected.append("Not Detected")
                    else:
                        Detected.append("Detected")
             
        if sum(fit) == False: #if all values are false and nothing fits then will append a 0. This makes sure you get the correct
                                  # number of values no matter what
                ProcessedX.append(0)  
                ProcessedY.append(0) #appends 0 for both 
                PPM.append(0)
                mzlistRep.append(MassNumbersList[mass])
                names.append(MassNumberNames[mass])  
                BlankRep.append(Blanks[mass])
                Detected.append("Not Detected")
        
 
        IndFile = pd.DataFrame(columns = ["Compound Names","mz_list",NameList+"_mass",NameList+"_count",NameList+"_PPM", "Blank STDev", "Detected"]) #columns mz_list, mass, and count +NameList[i]
        IndFile["Compound Names"]=names
        IndFile["mz_list"]=mzlistRep
        IndFile[NameList+"_mass"]=ProcessedX
        IndFile[NameList+"_count"]=ProcessedY #for this and the line before, it reads the values of processed from i*length
        IndFile[NameList+"_PPM"]=PPM
        IndFile["BlankSTDev"]=BlankRep
        IndFile["Detected"]=Detected
        IndFile.to_csv("/Users/timmo/Documents/RAPFLAB/code/code/blanktest/Processed/"+NameList+"_Processed.csv")          #chooses folder
    ####################


# AllFiles[outname+"mz_list"]=MassNumbersList   #Does same as for IndFile but for an All file with all values. Whole function works in similar way
# AllFiles[outname+"Compound Names"]=MassNumberNames
# for i in range(num_files):
#     AllFiles[outname+"_mass"+NameList[i]]=ProcessedX[(i*length):((i+1)*length)]
#     AllFiles[outname+"_count"+NameList[i]]=ProcessedY[(i*length):((i+1)*length)]
#     AllFiles[outname+"_PPM"+NameList[i]]=PPM[(i*length):((i+1)*length)]
        
# AllFiles.to_csv("/Users/timmo/Documents/RapfLab/code/MStest/List/Processed"+"/"+outname+"_AllProcessed.csv")

    

['210623_OOA_1hr_MS.csv', '210630_MeOHBlank_13_MS.csv', '210630_MeOHBlank_14_MS.csv']


FileNotFoundError: [Errno 2] No such file or directory: '/Users/timmo/Documents/RAPFLAB/code/code/blanktest/Processed/210623_OOA_1hr_MS_Processed.csv'